# Hours variable calculation

In [3]:
import os, sys, json, csv, ast, math, re
import numpy as np
import pandas as pd
from pandas import DataFrame
from ast import literal_eval
import datetime

In [7]:
path = "C:\\Users\\25982\\Dropbox\\STAT 628 Module 2 Yelp"
path_bus_train_expand= path+"\\bus_test_expand.csv"
print("Current working directory is: "+os.getcwd())

Current working directory is: C:\Users\25982\Desktop\M2


In [134]:
dat1 = pd.read_csv(path_bus_train_expand,index_col=0)
print(dat1.shape)

(38000, 56)

In [137]:
dat1[['Monday_open','Monday_close']]=dat1.pop('Monday').str.extract('(.*)-(.*)',  expand=True)
dat1[['Tuesday_open','Tuesday_close']]=dat1.pop('Tuesday').str.extract('(.*)-(.*)',  expand=True)
dat1[['Wednesday_open','Wednesday_close']]=dat1.pop('Wednesday').str.extract('(.*)-(Sunday.*)',  expand=True)
dat1[['Thursday_open','Thursday_close']]=dat1.pop('Thursday').str.extract('(.*)-(.*)',  expand=True)
dat1[['Friday_open','Friday_close']]=dat1.pop('Friday').str.extract('(.*)-(.*)',  expand=True)
dat1[['Saturday_open','Saturday_close']]=dat1.pop('Saturday').str.extract('(.*)-(.*)',  expand=True)
dat1[['Sunday_open','Sunday_close']]=dat1.pop('Sunday').str.extract('(.*)-(.*)',  expand=True)

In [140]:
week_day = ["Monday", "Tuesday" ,"Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
for temp_day in week_day:
    time_open = dat1[temp_day+'_open'].replace(np.nan, '', regex=True)
    time_open_datetime = (pd.Series ([datetime.datetime.strptime(item, "%H:%M") if item != '' else None
                                     for item in time_open ]))
    time_close = dat1[temp_day+'_close'].replace(np.nan, '', regex=True)
    time_close_datetime = (pd.Series ([datetime.datetime.strptime(item, "%H:%M") if item != '' else None
                                     for item in time_close ]))
    time_diff_1=(time_close_datetime-time_open_datetime)/np.timedelta64(1, 'h')
    time_diff_1[time_diff_1 <= 0]=time_diff_1[time_diff_1 <= 0]+24
    time_diff_1[time_diff_1.isnull() & dat1[['hours']].notnull().iloc[:,0]] = 0
    dat1[temp_day+"_hours"] = time_diff_1

In [141]:
dat1[["Sunday_open","Sunday_close", "Sunday_hours"]].head()

,Sunday_open,Sunday_close,Sunday_hours
0,9:0,0:0,15.0
1,NaN,NaN,0.0
2,NaN,NaN,0.0
3,11:0,18:0,7.0
4,9:0,21:0,12.0


In [142]:
for temp_day in week_day:
        dat1=dat1.drop(columns=[temp_day+'_open',temp_day+'_close'])

In [148]:
dat1.iloc[[1,2,6,7]]

,business_id,name,city,state,postal_code,latitude,longitude,is_open,attributes,categories,...,ByAppointmentOnly,WheelchairAccessible,RestaurantsTakeOut,Monday_hours,Tuesday_hours,Wednesday_hours,Thursday_hours,Friday_hours,Saturday_hours,Sunday_hours
1,148476,The UPS Store,Mississauga,ON,L5V 0B1,43.599475,-79.711584,1,"{'BusinessParking': ""{'garage': False, 'street...","Shipping Centers, Couriers & Delivery Services...",...,NaN,NaN,NaN,10.0,11.0,0,11.0,10.0,6.0,0.0
2,86835,Knot Salon,Scottsdale,AZ,85254,33.600071,-111.977371,1,"{'BusinessAcceptsCreditCards': 'True', 'Busine...","Hair Stylists, Beauty & Spas, Hair Salons, Men...",...,True,NaN,NaN,24.0,10.0,0,10.0,9.0,9.0,0.0
6,78312,Pine Village Apartments,Las Vegas,NV,89102,36.138452,-115.198019,1,NaN,"Home Services, Real Estate, Apartments",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,155769,Bruster's Real Ice Cream,Gilbert,AZ,85233,33.320386,-111.804652,0,"{'BusinessAcceptsCreditCards': 'True', 'BikePa...","Food, Desserts, Juice Bars & Smoothies, Ice Cr...",...,NaN,True,False,9.0,9.0,0,9.0,10.0,10.0,9.0


In [144]:
dat1.shape

(38000, 56)

In [147]:
dat1.to_csv('D:\\Projects\\Yelp\\Data\\bussiness_train_version3.csv')